<a href="https://colab.research.google.com/github/VadushaLyapushin/rzhavchina/blob/main/%D0%A1%D0%B2%D1%91%D1%80%D1%82%D0%BE%D1%87%D0%BD%D0%B0%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/VadushaLyapushin/rzhavchina.git

Cloning into 'rzhavchina'...
remote: Enumerating objects: 417, done.
remote: Counting objects: 100% (168/168), done.
remote: Compressing objects: 100% (165/165), done.
remote: Total 417 (delta 4), reused 165 (delta 3), pack-reused 249
Receiving objects: 100% (417/417), 76.24 MiB | 28.34 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense

In [3]:
# matplotlib.use("Agg")
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.optimizers import SGD
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import random
import pickle
import cv2
import os

from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

In [4]:
import matplotlib.pyplot as plt
import csv
import cv2
import numpy as np
import pickle
from skimage import exposure
import random

In [5]:
ImagePathsWithoot = list(paths.list_images("/content/rzhavchina/Transform Images/without"))
random.seed(4)
print(len(ImagePathsWithoot))
ImagePathsSurface = list(paths.list_images("/content/rzhavchina/Transform Images/surface"))
random.seed(4)
print(len(ImagePathsSurface))
ImagePathsDeep = list(paths.list_images("/content/rzhavchina/Transform Images/deep"))
random.seed(4)
print(len(ImagePathsDeep))

49
49
49


In [6]:
ImagePaths = ImagePathsWithoot + ImagePathsSurface + ImagePathsDeep
random.shuffle(ImagePaths)
print(len(ImagePaths))

147


In [26]:
data = []
labels = []

i=0
for imagepath in ImagePaths:
  image = cv2.imread(imagepath)
  image = cv2.resize(image, (32, 32))
  data.append(image)
  label = imagepath.split(os.path.sep)[-2]
  if label == "without":
    label = [1, 0, 0]
  elif label == "surface":
    label = [0, 1, 0]
  elif label == "deep":
    label = [0, 0, 1]
  labels.append(label)

print(labels)

data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

with open("/content/rzhavchina/training/data.pickle", 'wb') as f:
  pickle.dump(data, f)
print("Data seved")

with open("/content/rzhavchina/training/labels.pickle", 'wb') as f:
  pickle.dump(labels, f)
print("Labels seved")


[[0, 1, 0], [0, 0, 1], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 0, 1], [0, 1, 0], [0, 1, 0], [0, 0, 1], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 0, 1], [1, 0, 0], [0, 0, 1], [0, 0, 1], [1, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 0, 0], [1, 0, 0], [0, 0, 1], [1, 0, 0], [0, 0, 1], [1, 0, 0], [0, 1, 0], [0, 0, 1], [0, 1, 0], [1, 0, 0], [0, 0, 1], [0, 1, 0], [0, 0, 1], [0, 1, 0], [1, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1], [0, 1, 0], [0, 0, 1], [1, 0, 0], [0, 0, 1], [0, 1, 0], [0, 0, 1], [0, 1, 0], [1, 0, 0], [1, 0, 0], [0, 0, 1], [1, 0, 0], [0, 1, 0], [1, 0, 0], [0, 1, 0], [0, 1, 0], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 1, 0], [0, 0, 1], [1, 0, 0], [0, 1, 0], [1, 0, 0], [0, 1, 0], [0, 1, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 0, 1], [0, 0, 1], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [1, 0, 0], [0, 0, 1], [0, 0, 1], [0, 1, 0], [1, 0, 0], [1, 0, 0], [0, 1, 0], [0, 1, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1], [0, 1, 0], [0, 0, 1], [0, 1, 0], [0, 0, 1], [0, 1, 0]

In [27]:
np.prod(labels.shape)

441

In [28]:
with open("/content/rzhavchina/training/data.pickle", 'rb') as f:
  data = pickle.load(f)
print("Data loaded")

with open("/content/rzhavchina/training/labels.pickle", 'rb') as f:
  labels = pickle.load(f)
print("Labels loaded")

(trainX, testX, trainY, testY) = train_test_split(data, labels,
                                                  test_size=0.15,
                                                  random_state=42)

print("Dataset prepared")

Data loaded
Labels loaded
Dataset prepared


In [29]:
model = Sequential()
inputShape = (32, 32, 3)
chanDim = -1
    # CONV => RELU => BN => POOL
model.add(Conv2D(8, (5, 5), padding="same",
  input_shape=inputShape))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
  # first set of (CONV => RELU => CONV => RELU) * 2 => POOL
model.add(Conv2D(16, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(16, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
# second set of (CONV => RELU => CONV => RELU) * 2 => POOL
model.add(Conv2D(32, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(32, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
  # first set of FC => RELU layers
model.add(Flatten())
model.add(Dense(128))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))
# second set of FC => RELU layers
model.add(Flatten())
model.add(Dense(128))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))
# softmax classifier
model.add(Dense(3))
model.add(Activation("softmax"))
# return the constructed network architecture
print ("End")

End


In [30]:
import matplotlib
matplotlib.use("Agg")
# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
from skimage import transform
from skimage import exposure
from skimage import io
import matplotlib.pyplot as plt
import numpy as np
import random
import pickle
from sklearn.model_selection import train_test_split

In [48]:
with open("/content/rzhavchina/training/data.pickle", 'rb') as f:
  data = pickle.load(f)
print("Data loaded")

with open("/content/rzhavchina/training/labels.pickle", 'rb') as f:
  labels = pickle.load(f)
print("Labels loaded")

(trainX, testX, trainY, testY) = train_test_split(data, labels,
                                                  test_size=0.15,
                                                  random_state=42)

print("Dataset prepared")


# classTotals = trainY.sum(axis=0)
# classWeight = classTotals.max() / classTotals
# print(type(classWeight))

classTotals = trainY.sum(axis=0)
print(classTotals)
classWeight = classTotals.max() / classTotals
print(classWeight/1)

TestclassTotals = testY.sum(axis=0)
TestclassWeight = TestclassTotals.max() / TestclassTotals
print(TestclassTotals)
print("TestclassWeight")
print(TestclassWeight//1)
f=TestclassTotals.argmin()



# initialize the number of epochs to train for, base learning rate,
# and batch size
NUM_EPOCHS = 30
INIT_LR = 1e-3
BS = 32
# load the label names
target_names= ["without", "surface", "deep"]
numLabels = len(target_names)

# construct the image generator for data augmentation
opt = 'adam'
# model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# opt = keras.optimizers.legacy.Adam(lr=INIT_LR, decay=INIT_LR / (NUM_EPOCHS * 0.5))

# opt = tf.keras.optimizers.legacy.Adam(
#     learning_rate=INIT_LR,
#     beta_1=0.9,
#     beta_2=0.999,
#     epsilon=1e-07,
#     amsgrad=False,
#     name='Adam',
#     **kwargs)


model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])
model.summary()


Data loaded
Labels loaded
Dataset prepared
[42 41 41]
[1.         1.02439024 1.02439024]
[7 8 8]
TestclassWeight
[1. 1. 1.]
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 32, 32, 8)         608       
                                                                 
 activation_8 (Activation)   (None, 32, 32, 8)         0         
                                                                 
 batch_normalization_7 (Bat  (None, 32, 32, 8)         32        
 chNormalization)                                                
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 16, 16, 8)         0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 16, 16, 16)        1168   

In [49]:
aug = ImageDataGenerator(
	rotation_range=5,
	zoom_range=0.05,
	width_shift_range=0.05,
	height_shift_range=0.05,
	shear_range=0.10,
	horizontal_flip=False,
	vertical_flip=False,
	fill_mode="nearest")

# train the network
print("[INFO] training network...")
print(classWeight)

class_Weight = {0: 1.,
                1: 1.,
                2: 1.}


checkpointer = ModelCheckpoint(filepath='/content/rzhavchina/training/Best_Sign_Class.h5', verbose=1, save_best_only=True)

H = model.fit_generator(aug.flow(trainX, trainY, batch_size=BS),
												validation_data=(testX, testY),
												steps_per_epoch=trainX.shape[0] // BS,
												epochs=NUM_EPOCHS,
                        shuffle=True,
                        class_weight=class_Weight,
                        callbacks=[checkpointer])


[INFO] training network...
[1.         1.02439024 1.02439024]
Epoch 1/30


<ipython-input-49-5aa1e260e7a8>:22: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  H = model.fit_generator(aug.flow(trainX, trainY, batch_size=BS),


3/3 [==============================] - ETA: 0s - loss: 0.8027 - accuracy: 0.8043
Epoch 1: val_loss improved from inf to 3.00274, saving model to /content/rzhavchina/training/Best_Sign_Class.h5
3/3 [==============================] - 4s 278ms/step - loss: 0.8027 - accuracy: 0.8043 - val_loss: 3.0027 - val_accuracy: 0.2609


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 2/30
2/3 [===================>..........] - ETA: 0s - loss: 0.7946 - accuracy: 0.8125
Epoch 2: val_loss improved from 3.00274 to 2.69373, saving model to /content/rzhavchina/training/Best_Sign_Class.h5
3/3 [==============================] - 0s 119ms/step - loss: 0.8649 - accuracy: 0.8152 - val_loss: 2.6937 - val_accuracy: 0.3478
Epoch 3/30
2/3 [===================>..........] - ETA: 0s - loss: 0.6547 - accuracy: 0.8333
Epoch 3: val_loss did not improve from 2.69373
3/3 [==============================] - 0s 91ms/step - loss: 0.5444 - accuracy: 0.8587 - val_loss: 2.7647 - val_accuracy: 0.3043
Epoch 4/30
3/3 [==============================] - ETA: 0s - loss: 0.3232 - accuracy: 0.8854
Epoch 4: val_loss did not improve from 2.69373
3/3 [==============================] - 0s 86ms/step - loss: 0.3232 - accuracy: 0.8854 - val_loss: 2.8429 - val_accuracy: 0.4348
Epoch 5/30
2/3 [===================>..........] - ETA: 0s - loss: 0.3420 - accuracy: 0.8750
Epoch 5: val_loss did not improve fro